In [3]:
import numpy as np
import utils
import qiskit
import matplotlib.pyplot as plt
import pandas as pd
import dimod
from dwave.system import DWaveSampler
from dwave.system import EmbeddingComposite
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
provider = qiskit.IBMQ.load_account()

/home/user/.local/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


### Testing beta estimation function

In [44]:
def test_beta_estimation(qubits_n, betas):
    backend_sim = provider.get_backend('ibmq_qasm_simulator')
    for beta in betas:
        linear_interactions = {(i, i + 1): 1 for i in range(qubits_n - 1)}
        linear_circ = utils.linear_circuit(qubits_n, linear_interactions, beta=beta)
        linear_circ.measure_all()
        
        job_sim = qiskit.execute(linear_circ, backend_sim, shots=8192)
        sim_result = job_sim.result().get_counts()
        
        print(f'Real beta: {beta}. Estimation: {utils.estimate_beta(sim_result, linear_interactions)}')

In [42]:
test_beta_estimation(15, [0.01, 0.05, 0.1, 0.5, 1, 5])

Real beta: 0.01. Estimation: 0.06919862198472605
Real beta: 0.05. Estimation: -0.013488140055374109
Real beta: 0.1. Estimation: 0.099451099552261
Real beta: 0.5. Estimation: 0.21324308555587015
Real beta: 1. Estimation: 0.4690982441092791
Real beta: 5. Estimation: 3.6092105939121466


In [45]:
test_beta_estimation(5, [0.01, 0.05, 0.1, 0.5, 1, 5])

Real beta: 0.01. Estimation: 0.003493241298819949
Real beta: 0.05. Estimation: 0.05688275615594207
Real beta: 0.1. Estimation: 0.10176704684774522
Real beta: 0.5. Estimation: 0.4865860442150862
Real beta: 1. Estimation: 0.7994663165606339
Real beta: 5. Estimation: 4.158760998143585


### D-Wave beta on linear Ising models

In [53]:
def obtain_measurements_linear(qubit_ns):
    sampler = DWaveSampler() 
    embedding_sampler = EmbeddingComposite(sampler)
    for n in qubit_ns:
        bqm = dimod.BinaryQuadraticModel({i: 0 for i in range(n)},
                                 {(i, i + 1): -1 for i in range(n - 1)},
                                  0,
                                  dimod.Vartype.SPIN)
        sampleset = embedding_sampler.sample(bqm, num_reads=8192)
        d_wave_result = {}
        for (obs, _, oc, _) in sampleset.record:
            d_wave_result[utils.string01_from_spins(obs)] = oc
        beta = utils.estimate_beta(d_wave_result, {(i, i + 1): 1 for i in range(n - 1)})
        print(f'Chain length: {n}. Beta: {beta}')

In [54]:
obtain_measurements_linear([5, 15, 50, 100, 200, 500, 1000, 1500, 2048])

Chain length: 5. Beta: 1.977586505943844
Chain length: 15. Beta: 1.3898600414818656
Chain length: 50. Beta: 0.8296572539105571
Chain length: 100. Beta: 0.5514298528880175
Chain length: 200. Beta: 0.32978978182652796
Chain length: 500. Beta: -0.030517008018558573
Chain length: 1000. Beta: 0.03713288467285423
Chain length: 1500. Beta: -2.0185873175002847e-17
Chain length: 2048. Beta: -0.015518220460297272


### D-Wave beta on cyclic Ising models

In [56]:
def obtain_measurements_cyclic(qubit_ns):
    sampler = DWaveSampler() 
    embedding_sampler = EmbeddingComposite(sampler)
    for n in qubit_ns:
        bqm = dimod.BinaryQuadraticModel({i: 0 for i in range(n)},
                                 {(i, (i + 1) % n): -1 for i in range(n)},
                                  0,
                                  dimod.Vartype.SPIN)
        sampleset = embedding_sampler.sample(bqm, num_reads=8192)
        d_wave_result = {}
        for (obs, _, oc, _) in sampleset.record:
            d_wave_result[utils.string01_from_spins(obs)] = oc
        beta = utils.estimate_beta(d_wave_result, {(i, (i + 1) % n): 1 for i in range(n)})
        print(f'Cycle length: {n}. Beta: {beta}')

In [57]:
obtain_measurements_cyclic([5, 15, 50, 100, 200, 500, 1000, 1500, 2048])

Cycle length: 5. Beta: 2.0793804990717923
Cycle length: 15. Beta: 0.9605498060117768
Cycle length: 50. Beta: 0.5376321965175268
Cycle length: 100. Beta: 0.3884512903877117
Cycle length: 200. Beta: 0.04332169878499658
Cycle length: 500. Beta: 0.03095072908527117
Cycle length: 1000. Beta: -0.010830424696249147
Cycle length: 1500. Beta: 0.024968461106093406
Cycle length: 2048. Beta: -0.007220283130832748


### D-Wave beta on sqaure lattice Ising models

In [68]:
def make_square_lattice_interactions(n):
    def closest_factorization(n):
        h, w = None, None
        for d in range(1, int(n**(1/2)) + 1):
            if n % d == 0:
                h = d
                w = n // d
        return h, w
    
    def qubit_number(i, j, h, w):
        return w * i + j
    
    h, w = closest_factorization(n)
    interactions = {}
    for i in range(h):
        for j in range(w):
            cur = qubit_number(i, j, h, w)
            right = qubit_number(i, j + 1, h, w)
            bottom = qubit_number(i + 1, j, h, w)
            if j < w - 1:
                interactions[(cur, right)] = -1
            if i < h - 1:
                interactions[(cur, bottom)] = -1
    
    return interactions

In [73]:
def obtain_measurements_square_lattice(qubit_ns):
    sampler = DWaveSampler() 
    embedding_sampler = EmbeddingComposite(sampler)
    for n in qubit_ns:
        interactions = make_square_lattice_interactions(n)
        
        bqm = dimod.BinaryQuadraticModel({i: 0 for i in range(n)},
                                  interactions,
                                  0,
                                  dimod.Vartype.SPIN)
        sampleset = embedding_sampler.sample(bqm, num_reads=8192)
        d_wave_result = {}
        for (obs, _, oc, _) in sampleset.record:
            d_wave_result[utils.string01_from_spins(obs)] = oc
        beta = utils.estimate_beta(d_wave_result, {key: 1 for key, _ in interactions.items()})
        print(f'Lattice size: {n}. Beta: {beta}')

In [74]:
obtain_measurements_square_lattice([6, 15, 50, 100, 200, 500, 1000, 1500, 2048])

Lattice size: 6. Beta: 2.2526978173789005
Lattice size: 15. Beta: 1.1470844498088042
Lattice size: 50. Beta: 0.7552848865561808
Lattice size: 100. Beta: 0.5163560254016547
Lattice size: 200. Beta: 0.2519313198451169
Lattice size: 500. Beta: 0.01109709382493039
Lattice size: 1000. Beta: -0.001093840937814759
Lattice size: 1500. Beta: -0.009411917616573682
Lattice size: 2048. Beta: -0.01542639057964769
